<a href="https://colab.research.google.com/github/jihun-shim/study_NLPs/blob/main/codes/Deeplearings/10_04_GenerativeLLMwithHuggingface_evaluations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 제공된 압축 파일 풀기
- [chapter_6_withvLLMme_preprocess.zip](https://drive.google.com/file/d/1Kd7zJRRtQFxXNwaXZ2Ep60xFoO4h6kiU/view?usp=drive_link)
- [chapter_6_withvLLMme_finetuned_model.zip](https://drive.google.com/file/d/1ZGLkHmEigtNPBBtIn7GWwIA8jKlVJVps/view?usp=drive_link)

In [1]:
# !ls -al
!unzip chapter_6_withvLLMme_preprocess.zip
# !unzip chapter_6_withvLLMme_finetuned_model.zip

Archive:  chapter_6_withvLLMme_preprocess.zip
   creating: data/
  inflating: data/train.csv          
   creating: requests/
  inflating: requests/text2sql_evaluation.jsonl  
   creating: results/
  inflating: results/yi_ko_6b_eval.csv  
  inflating: results/text2sql_evaluation.jsonl  


## 자연어 평가
- EM(Exact Match) : 텍스트 매칭(정규식, 의미, 통계 매칭)
- EX(Executtion Accuracy) : 개발 환경에서 실행
- GPT-4 활용

In [2]:
!pip install transformers==4.40.1 bitsandbytes==0.43.1 accelerate==0.29.3 datasets==2.19.0 tiktoken==0.6.0 -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88

In [3]:
import os

In [4]:
import pandas as pd
df_dataset_text2sql_train = pd.read_csv('data/train.csv')
df_dataset_text2sql_train['text'].head(3)

,text
0,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
1,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
2,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...


In [5]:
df_dataset_text2sql_train.loc[1,'text'] # prompt 완료 명령어

'당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.\n\n### DDL:\nCREATE TABLE users (\n    "user_id" SERIAL PRIMARY KEY,\n    "email" VARCHAR(255) UNIQUE NOT NULL,\n    "password_hash" VARCHAR(255) NOT NULL,\n    "name" VARCHAR(255),\n    "created_at" TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,\n    "updated_at" TIMESTAMP\n);\nCREATE TABLE payment_methods (\n    "payment_method_id" SERIAL PRIMARY KEY,\n    "user_id" INTEGER REFERENCES users(user_id),\n    "card_number" VARCHAR(19),\n    "expiry_date" DATE,\n    "billing_address" TEXT,\n    "payment_type" VARCHAR(50)\n);\n\n### Question:\n카드로 결제한 사용자 중에 이름이 \'김\'씨씨인 사용자의 이메일과 카드 번호를 찾아주세요.\n\n### SQL:\nSELECT u.email, pm.card_number FROM users AS u JOIN payment_methods AS pm ON u.user_id = pm.user_id WHERE u.name LIKE \'김%\' AND pm.payment_type = \'카드\';'

## finetuning 된 모델 이용한 응답 만들기

In [6]:
import torch
from transformers import BitsAndBytesConfig, pipeline, AutoTokenizer, AutoModelForCausalLM

def make_inference_pipeline(model_id):
    # 토크나이징
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    # 모델
    model = AutoModelForCausalLM.from_pretrained(model_id
                                                , load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16 # 양자화 정의
                                                , device_map='auto')

    # pipline : 예측 초기화 설정
    pipe = pipeline('text-generation',model=model, tokenizer=tokenizer)
    return pipe

In [7]:
hf_finetunning_model_id = 'otter35/yi-ko-6b-text2sql'
hf_pipe = make_inference_pipeline(hf_finetunning_model_id)
hf_pipe

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.28M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [8]:
def make_prompt(ddl, question, query=''):
    prompt = f"""당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.
### DDL:
{ddl}
### Question:
{question}
### SQL:
{query}"""
    return prompt

In [9]:
# 필요한 컬럼 추출해 프롬프트 명령어 만들기
from tqdm import tqdm

df_total_count = len(df_dataset_text2sql_train)
for idx, row in tqdm(df_dataset_text2sql_train.iterrows(), total=df_total_count, desc='Generating prompts') : # DataFrame row 단위 loop
    # make_prompt(ddl, question, query='')
    prompt_command = make_prompt(ddl= row['context']
                                , question = row['question']
                                , query = '')
    df_dataset_text2sql_train.loc[idx, 'prompt'] = prompt_command
    pass

df_dataset_text2sql_train.head(3)

Generating prompts: 100%|██████████| 33876/33876 [00:07<00:00, 4708.21it/s]


,db_id,context,question,answer,text,prompt
0,3,"CREATE TABLE categories (\n ""category_id"" S...",카테고리 이름을 길이가 긴 순서로 정렬하되 가장 긴 이름 3개만 보여줘,SELECT name FROM categories ORDER BY LENGTH(na...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
1,4,"CREATE TABLE users (\n ""user_id"" SERIAL PRI...",카드로 결제한 사용자 중에 이름이 '김'씨씨인 사용자의 이메일과 카드 번호를 찾아주세요.,"SELECT u.email, pm.card_number FROM users AS u...",당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
2,4,"CREATE TABLE subscriptions (\n ""subscriptio...",시작일이 2022년 1월 1일 이후인 유효한 구독의 유저 아이디를 보여줘,SELECT DISTINCT user_id FROM subscriptions WHE...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...


## 파인튜닝된 모델에 prompt 질문과 답 얻기

In [14]:
example = df_dataset_text2sql_train.loc[1,'prompt']
example

'당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.\n### DDL:\nCREATE TABLE users (\n    "user_id" SERIAL PRIMARY KEY,\n    "email" VARCHAR(255) UNIQUE NOT NULL,\n    "password_hash" VARCHAR(255) NOT NULL,\n    "name" VARCHAR(255),\n    "created_at" TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,\n    "updated_at" TIMESTAMP\n);\nCREATE TABLE payment_methods (\n    "payment_method_id" SERIAL PRIMARY KEY,\n    "user_id" INTEGER REFERENCES users(user_id),\n    "card_number" VARCHAR(19),\n    "expiry_date" DATE,\n    "billing_address" TEXT,\n    "payment_type" VARCHAR(50)\n);\n### Question:\n카드로 결제한 사용자 중에 이름이 \'김\'씨씨인 사용자의 이메일과 카드 번호를 찾아주세요.\n### SQL:\n'

In [11]:
results = hf_pipe(example, do_sample=False
       , return_full_text=False, max_length=512, truncation=True)
results


[{'generated_text': "SELECT u.email, p.card_number FROM users AS u JOIN payment_methods AS p ON u.user_id = p.user_id WHERE u.name LIKE '%김%';"}]

In [17]:
results[0]['generated_text']

"SELECT u.email, p.card_number FROM users AS u JOIN payment_methods AS p ON u.user_id = p.user_id WHERE u.name LIKE '%김%';"

In [19]:
# results_gen_sqls = hf_pipe(df_dataset_text2sql_train['prompt'].tolist(), do_sample=False
#       #  , return_full_text=False, max_length=50, truncation=True)
#        , return_full_text=False, max_new_tokens=50, truncation=True)

results_gen_sqls = []
for prompt in tqdm(df_dataset_text2sql_train['prompt'].tolist(), desc="Generating SQL"):
    sql = hf_pipe([prompt], do_sample=False, return_full_text=False, max_length=512, truncation=True)
    results_gen_sqls.append(sql[0][0]['generated_text'])

df_dataset_text2sql_train['gen_sql'] = results_gen_sqls
df_dataset_text2sql_train.head(3)

Generating SQL:   0%|          | 20/33876 [00:48<22:42:12,  2.41s/it]


KeyboardInterrupt: 

### 평가 위한 프롬포트 작성

In [ ]:
import json
import pandas as pd
from pathlib import Path

# gpt_model = "gpt-4-turbo-preview"
gpt_model = "gpt-4o-mini"
def make_requests_for_gpt_evaluation(df, filename, dir='requests'):
  if not Path(dir).exists():
      Path(dir).mkdir(parents=True)
  prompts = []
  for idx, row in df.iterrows():
      prompts.append("""Based on below DDL and Question, evaluate gen_sql can resolve Question. If gen_sql and gt_sql do equal job, return "yes" else return "no". Output JSON Format: {"resolve_yn": ""}""" + f"""
          DDL: {row['context']}
          Question: {row['question']}
          gt_sql: {row['answer']}
          gen_sql: {row['gen_sql']}"""
          )

  jobs = [{"model": gpt_model, "response_format" : { "type": "json_object" }, "messages": [{"role": "system", "content": prompt}]} for prompt in prompts]
  with open(Path(dir, filename), "w") as f:
      for job in jobs:
          json_string = json.dumps(job)
          f.write(json_string + "\n")

In [ ]:
# 프롬프트 일괄 작성 위한 function 호출과 파일 작성(parallel하게 openapi 호출 위해)
make_requests_for_gpt_evaluation(df_dataset_text2sql_train,'text2sql_evaluation.jsonl')

In [20]:
!wget https://raw.githubusercontent.com/openai/openai-cookbook/refs/heads/main/examples/api_request_parallel_processor.py

--2025-02-10 08:09:45--  https://raw.githubusercontent.com/openai/openai-cookbook/refs/heads/main/examples/api_request_parallel_processor.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21891 (21K) [text/plain]
Saving to: ‘api_request_parallel_processor.py’

api_request_paralle 100%[===================>]  21.38K  --.-KB/s    in 0.001s  

2025-02-10 08:09:45 (15.3 MB/s) - ‘api_request_parallel_processor.py’ saved [21891/21891]



In [ ]:
!mkdir results

In [23]:
# GPT-4 평가 수행
eval_filepath = 'text2sql_evaluation.jsonl'

!python api_request_parallel_processor.py \
--requests_filepath requests/{eval_filepath}  \
--save_filepath results/{eval_filepath} \
--request_url https://api.openai.com/v1/chat/completions \
--max_requests_per_minute 2500 \
--max_tokens_per_minute 100000 \
--token_encoding_name cl100k_base \
--max_attempts 5 \
--logging_level 20

INFO:root:Starting request #0
INFO:root:Starting request #1
INFO:root:Starting request #2
INFO:root:Starting request #3
INFO:root:Starting request #4
INFO:root:Starting request #5
INFO:root:Starting request #6
INFO:root:Starting request #7
INFO:root:Starting request #8
INFO:root:Starting request #9
INFO:root:Starting request #10
INFO:root:Starting request #11
INFO:root:Starting request #12
INFO:root:Starting request #13
INFO:root:Starting request #14
INFO:root:Starting request #15
INFO:root:Starting request #16
INFO:root:Starting request #17
INFO:root:Starting request #18
INFO:root:Starting request #19
INFO:root:Starting request #20
INFO:root:Starting request #21
INFO:root:Starting request #22
INFO:root:Starting request #23
INFO:root:Starting request #24
INFO:root:Starting request #25
INFO:root:Starting request #26
INFO:root:Starting request #27
INFO:root:Starting request #28
INFO:root:Starting request #29
INFO:root:Starting request #30
INFO:root:Starting request #31
INFO:root:Starting

### openAI 결과(results/text2sql_evaluation.jsonl)를 DataFrame 변환

In [ ]:
def change_jsonl_to_csv(input_file, output_file, prompt_column="prompt", response_column="response"):
    prompts = []
    responses = []
    with open(input_file, 'r') as json_file:
        for data in json_file:
            prompts.append(json.loads(data)[0]['messages'][0]['content'])
            responses.append(json.loads(data)[1]['choices'][0]['message']['content'])
    df = pd.DataFrame({prompt_column: prompts, response_column: responses})
    df.to_csv(output_file, index=False)
    return df

In [ ]:
df_evalations = change_jsonl_to_csv(input_file = 'results/text2sql_evaluation.jsonl'
                    ,output_file = 'results/yi_ko_6b_eval.csv'
                    ,prompt_column='prompt'
                    ,response_column='resolve_yn')

In [ ]:
# 실제값 josn : {"resolve_yn" : "no"}

df_evalations['resolve_yn'] = df_evalations['resolve_yn'].apply(lambda x: json.loads(x)['resolve_yn'])
correnct_sql_number = df_evalations.query("resolve_yn" == "yes").shape[0]
correnct_sql_number # 평가 맞은 갯수